Setup OAuth for accessing PALM finetuned model API

In [ ]:
!pip install --upgrade -q google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install google-generativeai

In [2]:
# Install the client library and import necessary modules.
# !pip install google-generativeai
import google.generativeai as palm
import base64
import json
import pprint

C:\Users\yamup\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/generative-language.tuning']

def load_creds():
    """Converts `oauth-client-id.json` to a credential object.

    This function caches the generated tokens to minimize the use of the
    consent screen.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'oauth-client-id.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

In [4]:
import pprint
import google.generativeai as palm

creds = load_creds()

palm.configure(credentials=creds)

print('Available base models:', [m.name for m in palm.list_models()])
print('My tuned models:', [m.name for m in palm.list_tuned_models()])

Available base models: ['models/chat-bison-001', 'models/text-bison-001', 'models/embedding-gecko-001', 'models/gemini-pro', 'models/gemini-pro-vision', 'models/embedding-001', 'models/aqa']
My tuned models: ['tunedModels/adobemid-fwdqq210yvfu', 'tunedModels/palmfinetuningapproach3-kdvh8t1tct5a']


In [5]:
model_name = 'tunedModels/adobemid-fwdqq210yvfu'
model = palm.get_model(model_name)

In [6]:
model = model_name 
temperature = 0 
candidate_count = 1 
top_k = 40 
top_p = 0.95 
max_output_tokens = 1024 

defaults = {
  'model': model,
  'temperature': temperature,
  'candidate_count': candidate_count,
  'top_k': top_k,
  'top_p': top_p,
  'max_output_tokens': max_output_tokens,
}

In [16]:
def get_response(prompt, defaults):
    """Returns the response from the model."""
    response = palm.generate_text(**defaults,prompt=prompt)
    
    if len(response.candidates) == 0:
        return ' '
    return response.candidates[0]['output']

Test Approach 1

In [8]:
prompt_template = "Context: The tweet was posted by '{company}' on '{date}' having '{like}' likes. The tweet contains the image with the following description - '{media_desc}'. The username of the company is '{username}'. Question: What is the tweet that was written based on the given information?"

In [17]:
import numpy as np
import pandas as pd
test_tweet = np.load('../data/tweet_desc.npy', allow_pickle=True)[1000:2000]
test_likes = np.load('../data/tweet_likes.npy', allow_pickle=True)[1000:2000]

generated_and_actual = []

for i in range(len(test_tweet)):
    try:
        tweet = test_tweet[i]
        like = test_likes[i]
        company = tweet['company']
        date = tweet['date']
        media_desc = tweet['media']
        username = tweet['username']

        prompt = prompt_template.format(company=company, date=date, like=like, media_desc=media_desc, username=username)
        generated_response = get_response(prompt, defaults)


        generated_and_actual.append({
            'generated': generated_response,
            'actual': tweet['content']
        })
    except Exception as e:
        print(f"Error: {e}. index: {i}")
        
df = pd.DataFrame(generated_and_actual)
df.to_csv('../saved/generated_and_actual_appr_1.csv', index=False)

In [20]:
df = pd.read_csv('../saved/generated_and_actual_appr_1.csv')

for i in range(len(df)):
    item = df.loc[i, 'generated']
    if item.startswith('Answer:') or item.startswith('Solution:'):
        df.loc[i, 'generated'] = item.split(':')[1].strip()

df.to_csv('../saved/generated_and_actual_appr_1_updated.csv', index=False)
